This assignment uses uses the MNL model to analyze (1) yogurt purchase data made by consumers at a retail location, and (2) conjoint data about consumer preferences for minivans.


## 1. Estimating Yogurt Preferences

### Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 4 products, then either $y=3$ or $y=(0,0,1,0)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, size, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 4 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta} + e^{x_4'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=\delta_{i4}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 \times \mathbb{P}_i(4)^0 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$


### Yogurt Dataset

We will use the `yogurt_data` dataset, which provides anonymized consumer identifiers (`id`), a vector indicating the chosen product (`y1`:`y4`), a vector indicating if any products were "featured" in the store as a form of advertising (`f1`:`f4`), and the products' prices (`p1`:`p4`). For example, consumer 1 purchased yogurt 4 at a price of 0.079/oz and none of the yogurts were featured/advertised at the time of consumer 1's purchase.  Consumers 2 through 7 each bought yogurt 2, etc.

_todo: import the data, maybe show the first few rows, and describe the data a bit._


In [ ]:
import pandas as pd

yogurt_data = pd.read_csv('yogurt_data.csv')

print(yogurt_data.head())
print(yogurt_data.describe())

Let the vector of product features include brand dummy variables for yogurts 1-3 (we'll omit a dummy for product 4 to avoid multi-collinearity), a dummy variable to indicate if a yogurt was featured, and a continuous variable for the yogurts' prices:  

$$ x_j' = [\mathbbm{1}(\text{Yogurt 1}), \mathbbm{1}(\text{Yogurt 2}), \mathbbm{1}(\text{Yogurt 3}), X_f, X_p] $$

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). 

What we would like to do is reorganize the data from a "wide" shape with $n$ rows and multiple columns for each covariate, to a "long" shape with $n \times J$ rows and a single column for each covariate.  As part of this re-organization, we'll add binary variables to indicate the first 3 products; the variables for featured and price are included in the dataset and simply need to be "pivoted" or "melted" from wide to long.  

_todo: reshape and prep the data_


In [ ]:
# Reshape the data to a long format
long_format = pd.melt(yogurt_data, id_vars=['id'],
                      value_vars=['y1', 'y2', 'y3', 'y4', 'f1', 'f2', 'f3', 'f4', 'p1', 'p2', 'p3', 'p4'],
                      var_name='product', value_name='value')

# Split the product column to separate type and product number
long_format['type'] = long_format['product'].str[0]  # 'y', 'f', or 'p'
long_format['product'] = long_format['product'].str[1:].astype(int)  # 1, 2, 3, or 4

# Pivot the table to get 'chosen', 'featured', and 'price' as columns
reshaped_data = long_format.pivot_table(index=['id', 'product'], columns='type', values='value', aggfunc='first').reset_index()

# Create dummy variables for the first three products
reshaped_data['yogurt1'] = (reshaped_data['product'] == 1).astype(int)
reshaped_data['yogurt2'] = (reshaped_data['product'] == 2).astype(int)
reshaped_data['yogurt3'] = (reshaped_data['product'] == 3).astype(int)

# Rename columns for clarity
reshaped_data.rename(columns={'y': 'chosen', 'f': 'featured', 'p': 'price'}, inplace=True)

# Display the first few rows of the reshaped data
print(reshaped_data.head())

### Estimation

_todo: Code up the log-likelihood function._


In [ ]:
import numpy as np

def log_likelihood(beta, data):
    """
    Calculate the log-likelihood of the Multi-nomial Logit model.

    Parameters:
    - beta: numpy array of coefficients for the utility functions.
    - data: pandas DataFrame with the reshaped yogurt data.

    Returns:
    - The log-likelihood value.
    """
    # Calculate utilities for each product and each choice
    # Assume 'data' has columns 'yogurt1', 'yogurt2', 'yogurt3', 'featured', 'price' for the covariates
    X = data[['yogurt1', 'yogurt2', 'yogurt3', 'featured', 'price']]
    utilities = X.dot(beta)
    
    # Reshape utilities to have one row per consumer, assuming the same number of rows per consumer
    num_products = 4  # Number of product choices
    utilities = utilities.values.reshape(-1, num_products)
    
    # Compute the softmax probabilities for each choice
    max_utilities = np.max(utilities, axis=1, keepdims=True)
    exp_utilities = np.exp(utilities - max_utilities)
    choice_probabilities = exp_utilities / np.sum(exp_utilities, axis=1, keepdims=True)
    
    # Get the choice indicator matrix
    chosen = data['chosen'].values.reshape(-1, num_products)
    
    # Calculate the log-likelihood
    log_probs = np.log(choice_probabilities) * chosen
    log_likelihood = np.sum(log_probs)
    
    return log_likelihood

# Example usage:
beta_initial = np.zeros(5)  # Initialize beta values (5 coefficients for the features)
log_likelihood_value = log_likelihood(beta_initial, reshaped_data)
print("Log-Likelihood:", log_likelihood_value)

_todo: Use `optim()` in R or `optimize()` in Python to find the MLEs for the 5 parameters ($\beta_1, \beta_2, \beta_3, \beta_f, \beta_p$).  (Hint: you should find 2 positive and 1 negative product intercepts, a small positive coefficient estimate for featured, and a large negative coefficient estimate for price.)_


In [ ]:
from scipy.optimize import minimize
import numpy as np

def negative_log_likelihood(beta, data):
    """
    Calculate the negative log-likelihood for the MNL model to be used in optimization.
    This is the negative of the log_likelihood function defined earlier.
    """
    X = data[['yogurt1', 'yogurt2', 'yogurt3', 'featured', 'price']]
    utilities = X.dot(beta)
    num_products = 4
    utilities = utilities.values.reshape(-1, num_products)
    
    max_utilities = np.max(utilities, axis=1, keepdims=True)
    exp_utilities = np.exp(utilities - max_utilities)
    choice_probabilities = exp_utilities / np.sum(exp_utilities, axis=1, keepdims=True)
    
    chosen = data['chosen'].values.reshape(-1, num_products)
    
    log_probs = np.log(choice_probabilities) * chosen
    log_likelihood = np.sum(log_probs)
    
    return -log_likelihood  # Return negative log-likelihood

# Initial guess for the parameters: [beta1, beta2, beta3, beta_f, beta_p]
initial_guess = np.zeros(5)

# Perform the minimization using 'BFGS' algorithm
result = minimize(negative_log_likelihood, initial_guess, args=(reshaped_data,), method='BFGS')

print("Optimization Results:")
print("Estimated coefficients:", result.x)
print("Success:", result.success)
print("Message:", result.message)

### Discussion

We learn...

_todo: interpret the 3 product intercepts (which yogurt is most preferred?)._


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example estimated intercepts from your optimization results
beta_estimates = np.array([0.5, 0.2, -0.1])  # Replace these with your actual results
product_labels = ['Yogurt 1', 'Yogurt 2', 'Yogurt 3']

# Optional: If you have standard errors, you can calculate confidence intervals
# Example standard errors for each beta
standard_errors = np.array([0.05, 0.05, 0.05])  # Placeholder values
confidence_intervals = 1.96 * standard_errors  # 95% CI

# Create a bar plot
plt.figure(figsize=(8, 5))
plt.bar(product_labels, beta_estimates, yerr=confidence_intervals, color='skyblue', capsize=5)
plt.ylabel('Coefficient Estimate')
plt.title('Intercept Coefficients for Yogurts')
plt.axhline(0, color='gray', linewidth=0.8)  # Draw a line at zero for reference

# Display the plot
plt.show()

_todo: use the estimated price coefficient as a dollar-per-util conversion factor. Use this conversion factor to calculate the dollar benefit between the most-preferred yogurt (the one with the highest intercept) and the least preferred yogurt (the one with the lowest intercept). This is a per-unit monetary measure of brand value._


In [ ]:
# Coefficients from the model
beta_intercepts = np.array([0.5, 0.2, -0.1])  # Replace these with your actual results
beta_price = -1.0  # Replace with your actual price coefficient

# Find the indices for the most and least preferred yogurts
index_most_preferred = np.argmax(beta_intercepts)
index_least_preferred = np.argmin(beta_intercepts)

# Calculate the utility difference
utility_difference = beta_intercepts[index_most_preferred] - beta_intercepts[index_least_preferred]

# Convert utility difference to dollar benefit using the price coefficient
# Note: Since beta_price is negative, we use its absolute value to convert utilities to dollars
dollar_benefit = utility_difference / abs(beta_price)

print(f"Dollar benefit of the most preferred over the least preferred yogurt: ${dollar_benefit:.2f}")

One benefit of the MNL model is that we can simulate counterfactuals (eg, what if the price of yogurt 1 was $0.10/oz instead of $0.08/oz).

_todo: calculate the market shares in the market at the time the data were collected.  Then, increase the price of yogurt 1 by $0.10 and use your fitted model to predict p(y|x) for each consumer and each product (this should be a matrix of $N \times 4$ estimated choice probabilities.  Take the column averages to get the new, expected market shares that result from the $0.10 price increase to yogurt 1.  Do the yogurt 1 market shares decrease?_




## 2. Estimating Minivan Preferences


### Data

_todo: download the dataset from here:_ http://goo.gl/5xQObB 


In [ ]:
import requests

# URL of the dataset
url = 'http://goo.gl/5xQObB'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open a file to write the dataset to (specify the path and file name)
    with open('minivan_data.csv', 'wb') as file:
        file.write(response.content)
    print("Dataset downloaded successfully.")
else:
    print("Failed to download the dataset. Status code:", response.status_code)

_todo: describe the data a bit. How many respondents took the conjoint survey?  How many choice tasks did each respondent complete?  How many alternatives were presented on each choice task? For each alternative._


In [ ]:
minivan_data = pd.read_csv('minivan_data.csv')

print(minivan_data.head())
print(minivan_data.describe())
print(minivan_data.info())

# Assuming there is a column that identifies each respondent
respondent_count = minivan_data['respondent_id'].nunique()
print(f"Number of respondents: {respondent_count}")

# Assuming there is a column that identifies each choice task
tasks_per_respondent = minivan_data.groupby('respondent_id')['task_id'].nunique()
print(f"Number of choice tasks completed by each respondent (example for one respondent): {tasks_per_respondent.iloc[0]}")

# Assuming there is a column that details the alternatives presented in each task
alternatives_per_task = minivan_data.groupby(['respondent_id', 'task_id']).size()
print(f"Number of alternatives presented on each choice task (example for one task): {alternatives_per_task.iloc[0]}")

The attributes (levels) were number of seats (6,7,8), cargo space (2ft, 3ft), engine type (gas, hybrid, electric), and price (in thousands of dollars).


### Model

_todo: estimate a MNL model omitting the following levels to avoide multicollinearity (6 seats, 2ft cargo, and gas engine). Include price as a continuous variable. Show a table of coefficients and standard errors.  You may use your own likelihood function from above, or you may use a function from a package/library to perform the estimation._  


In [ ]:
import statsmodels.api as sm

data = pd.read_csv('minivan_data.csv')  # adjust path as necessary

data['seats_7'] = (data['seat'] == 7).astype(int)
data['seats_8'] = (data['seat'] == 8).astype(int)
data['cargo_3ft'] = (data['cargo'] == '3ft').astype(int)
data['engine_hybrid'] = (data['eng'] == 'hyb').astype(int)
data['engine_electric'] = (data['eng'] == 'elec').astype(int)

# The dependent variable 'choice' should be binary indicating the choice made
X = data[['seats_7', 'seats_8', 'cargo_3ft', 'engine_hybrid', 'engine_electric', 'price']]
y = data['choice']  # binary choice

# Add a constant to the independent variables
X = sm.add_constant(X)

# Fit the logistic regression model
model = sm.Logit(y, X)
result = model.fit()

# Display the results
print(result.summary())

### Results

_todo: Interpret the coefficients. Which features are more preferred?_

The const represents the baseline utility for choosing a minivan. Which are seats_6, cargo_2ft, and engine_gas. The coefficient for seats 7 and 8 was negative, stating that 6 seats is more preferred. 

The base line for cargo space is 2ft. The coefficient for 3ft cargo was a positive stating 3ft cargo space is the most preferred option.

The coefficient for electric and hybrid was negative, representing that the baseline of Gas engine is the most preferred option. 

The price coefficient is negative, indicating that respondents prefer lower prices. 

_todo: Use the price coefficient as a dollar-per-util conversion factor. What is the dollar value of 3ft of cargo space as compared to 2ft of cargo space?_


In [ ]:
beta_cargo_3ft = 0.4385
beta_price = -0.1591

# Calculate the dollar value of 3ft of cargo space
dollar_value_cargo_space = beta_cargo_3ft / abs(beta_price)
print("Dollar value of 3ft cargo space compared to 2ft: $", dollar_value_cargo_space)

_todo: assume the market consists of the following 6 minivans. Predict the market shares of each minivan in the market._

| Minivan | Seats | Cargo | Engine | Price |
|---------|-------|-------|--------|-------|
| A       | 7     | 2     | Hyb    | 30    |
| B       | 6     | 2     | Gas    | 30    |
| C       | 8     | 2     | Gas    | 30    |
| D       | 7     | 3     | Gas    | 40    |
| E       | 6     | 2     | Elec   | 40    |
| F       | 7     | 2     | Hyb    | 35    |


_hint: this example is taken from the "R 4 Marketing Research" book by Chapman and Feit. I believe the same example is present in the companion book titled "Python 4 Marketing Research".  I encourage you to attempt these questions on your own, but if you get stuck or would like to compare you results to "the answers," you may consult the Chapman and Feit books._


In [ ]:
import numpy as np
import pandas as pd

# Define the minivan attributes
minivans = pd.DataFrame({
    'Minivan': ['A', 'B', 'C', 'D', 'E', 'F'],
    'seats': [7, 6, 8, 7, 6, 7],
    'cargo': [2, 2, 2, 3, 2, 2],
    'engine': ['hyb', 'gas', 'gas', 'gas', 'elec', 'hyb'],
    'price': [30, 30, 30, 40, 40, 35]
})

# Map the attributes to the model features
minivans['seats_7'] = (minivans['seats'] == 7).astype(int)
minivans['seats_8'] = (minivans['seats'] == 8).astype(int)
minivans['cargo_3ft'] = (minivans['cargo'] == 3).astype(int)
minivans['engine_hybrid'] = (minivans['engine'] == 'hyb').astype(int)
minivans['engine_electric'] = (minivans['engine'] == 'elec').astype(int)

# Coefficients from the logistic regression model
coefficients = {
    'const': 5.5322,
    'seats_7': -0.5248,
    'seats_8': -0.2931,
    'cargo_3ft': 0.4385,
    'engine_hybrid': -0.7605,
    'engine_electric': -1.4347,
    'price': -0.1591
}

# Calculate utility for each minivan
minivans['utility'] = (coefficients['const'] +
                       coefficients['seats_7'] * minivans['seats_7'] +
                       coefficients['seats_8'] * minivans['seats_8'] +
                       coefficients['cargo_3ft'] * minivans['cargo_3ft'] +
                       coefficients['engine_hybrid'] * minivans['engine_hybrid'] +
                       coefficients['engine_electric'] * minivans['engine_electric'] +
                       coefficients['price'] * minivans['price'])

# Calculate the choice probabilities using the softmax function
minivans['exp_utility'] = np.exp(minivans['utility'])
sum_exp_utility = minivans['exp_utility'].sum()
minivans['probability'] = minivans['exp_utility'] / sum_exp_utility

# Display the market shares (choice probabilities)
minivan_shares = minivans[['Minivan', 'probability']]
minivan_shares.columns = ['Minivan', 'Market Share']

print(minivan_shares)